# Análise Exploratória Avançada dos Dados

Este notebook tem como objetivo realizar uma análise exploratória e pré-processamento dos dados operacionais da Companhia Paulista de Trens Metropolitanos (CPTM). Utilizando técnicas de ciência de dados e aprendizado de máquina, o projeto visa identificar padrões de movimentação de passageiros, otimizar a oferta de transporte e melhorar a eficiência operacional.

A análise será dividida em duas principais seções, cada uma correspondendo a um dos datasets fornecidos:

- Movimentação de Validações (df_mov_periodo): Este dataset contém informações sobre a validação de bilhetes, detalhando o número de validações por estação, horário e tipo de dia (dia útil ou final de semana). Ele será utilizado para identificar padrões de uso do sistema ferroviário, detectar anomalias e momentos de alta ou baixa demanda, além de explorar a relação entre variáveis, como o tipo de dia e o total de validações.

- Tipos de Embarque (df_embarque): Este dataset contém registros sobre os tipos de bilhetes utilizados pelos passageiros (pagante, gratuito, bloqueado, etc.). A análise desse conjunto de dados busca compreender o perfil dos usuários da CPTM, identificar os tipos de bilhetes mais utilizados e como isso impacta a operação da companhia. Além disso, será realizado um estudo para avaliar a participação dos passageiros de baixa renda ou com gratuidade nos embarques, auxiliando no planejamento tarifário e nas políticas públicas.

## Importação de bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

## Análise de Movimentação de Validações

- O dataset registra a movimentação total de validações de bilhetes por período, incluindo o número de validações em uma determinada estação, horário e tipo de dia (ex: dia útil, final de semana). Ele pode ser utilizado para análise da quantidade de passageiros em períodos específicos.

In [ ]:
# Classe para análise de Validações
class ValidationAnalysis:
    def __init__(self, file_path):
        self.df_mov_periodo = pd.read_csv(file_path, sep=',')
        self.df_mov_periodo.columns = self.df_mov_periodo.columns.str.replace('"', '')

    def preprocess_data(self):
        self.df_mov_periodo['DT_VALIDACAO'] = pd.to_datetime(self.df_mov_periodo['DT_VALIDACAO'], errors='coerce')
        self.df_mov_periodo.dropna(subset=['DT_VALIDACAO'], inplace=True)
        self.df_mov_periodo['TOTAL_VALIDACOES'] = pd.to_numeric(self.df_mov_periodo['TOTAL_VALIDACOES'], errors='coerce')
        self.df_mov_periodo.dropna(subset=['TOTAL_VALIDACOES'], inplace=True)
        self.df_mov_periodo['TIPO_DIA'] = self.df_mov_periodo['TIPO_DIA'].replace({'U': -1, 'D': 0, 'F': 1})
        self.df_mov_periodo['TIPO_DIA'] = pd.to_numeric(self.df_mov_periodo['TIPO_DIA'], errors='coerce')
        self.df_mov_periodo.dropna(subset=['TIPO_DIA'], inplace=True)

        print("Dados de Validações Após Preprocessamento:")
        print(self.df_mov_periodo.head())
        
        numero_colunas = self.df_mov_periodo    .shape[1]
        print(f"O dataset tem {numero_colunas} colunas.")

    def univariate_analysis(self):
        plt.figure(figsize=(10,6))
        self.df_mov_periodo['TOTAL_VALIDACOES'].plot(kind='hist', bins=20, color='blue', alpha=0.7)
        plt.title('Distribuição do Total de Validações')
        plt.xlabel('Total de Validações')
        plt.ylabel('Frequência')
        plt.show()

    def bivariate_analysis(self):
        heatmap_data = self.df_mov_periodo.pivot_table(index='DT_VALIDACAO', columns='CD_ESTAC_BU', values='TOTAL_VALIDACOES', aggfunc='sum')

        plt.figure(figsize=(12, 8))
        sns.heatmap(heatmap_data, cmap='Blues')
        plt.title('Mapa de Calor - Validações por Estação e Data')
        plt.xlabel('Estação')
        plt.ylabel('Data')
        plt.show()

    def pca_analysis(self):
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(self.df_mov_periodo[['TOTAL_VALIDACOES', 'TIPO_DIA']])
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(scaled_data)

        plt.figure(figsize=(8,6))
        plt.scatter(pca_result[:,0], pca_result[:,1], c=self.df_mov_periodo['TOTAL_VALIDACOES'], cmap='viridis', alpha=0.5)
        plt.colorbar(label='Total Validações')
        plt.title('PCA - Validações')
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.show()
        
    def return_df(self):
        return self.df_mov_periodo

In [ ]:
validation_analysis = ValidationAnalysis('DMO_ANL_VW_TOT_MOV_PERIODO.csv')

### Pré-processamento

A primeira etapa no processamento de dados é garantir que as informações estão corretamente carregadas no ambiente para que possam ser manipuladas e analisadas.

In [ ]:
validation_analysis.preprocess_data()

### Análise Univariada

A coluna TOTAL_VALIDACOES reflete o número de validações em cada estação e período. A visualização da distribuição dessa coluna ajuda a entender a frequência de validações e detectar possíveis picos ou queda de movimentação em determinados horários ou dias.

In [ ]:
validation_analysis.univariate_analysis()

- Hipótese: A maioria dos períodos de validação tem apenas uma validação registrada, indicando baixa movimentação em determinados horários e estações.

-  O gráfico demonstra que a maioria dos períodos possui apenas 1 validação, com poucos registros de 2 ou 3 validações. Isso sugere uma subutilização do sistema em horários específicos ou uma possível falha no sistema de bilhetagem que precisa ser investigada.

- O sistema de bilhetagem da CPTM pode estar falhando em capturar todas as validações, ou a demanda por transporte é realmente baixa em determinados horários. Para otimizar os recursos, a CPTM pode considerar ajustes na frequência dos trens fora dos horários de pico, adaptando o número de trens e o pessoal operacional durante esses horários.

### Análise Bivariada

Colunas utilizadas: ID_DT_HORA_MINUTO, COD_BILH, CD_ESTAC_BU, DT_VALIDACAO, TOTAL_VALIDACOES, TIPO_DIA.
Essas colunas são selecionadas porque permitem identificar relações entre o tempo, as estações e o número de validações. O objetivo é entender como essas variáveis se correlacionam e se há padrões claros, como mais validações em certas estações ou períodos específicos.

In [ ]:
validation_analysis.bivariate_analysis()

- Hipótese: Algumas estações apresentam um maior volume de validações em dias específicos, indicando padrões concentrados de uso em certos períodos.

- Análise: O mapa de calor revela que estações como 504, 506, e 663 têm picos de validações em datas específicas, sugerindo maior movimentação nessas áreas, enquanto outras estações mostram baixa ou nenhuma validação.

- Conclusão: A CPTM pode otimizar sua operação focando em estações e períodos de alta demanda, ajustando a alocação de trens e recursos de forma eficiente, além de usar essas informações para prever picos futuros e planejar melhorias.

### Análise de Componentes Principais (PCA)

A análise de componentes principais (PCA) é usada para reduzir a dimensionalidade dos dados e verificar se as principais variáveis (nesse caso, o tipo de dia e o total de validações) são suficientes para explicar a variabilidade no número de validações. Isso ajuda a identificar os fatores mais importantes que afetam o número de validações.

In [ ]:
validation_analysis.pca_analysis()

- Hipótese: As componentes principais irão revelar que o total de validações é influenciado principalmente pelo tipo de dia e outros fatores.

- Análise: O PCA mostra uma clara distinção entre os períodos com base nas validações, sugerindo que variáveis como tipo de dia e total de validações influenciam significativamente os padrões de uso.

- Conclusão: A análise de PCA sugere que a CPTM pode usar essas variáveis principais para prever padrões de demanda e ajustar o serviço de forma preditiva, com base no histórico de uso de validações.

## Análises do Dataset de Embarque

- O dataset contém informações sobre os tipos de embarque, com identificadores de bilhetes, descrições dos movimentos (ex: idoso, deficiente) e um código de lançamento associado a cada tipo. Esses dados são úteis para categorizar e monitorar os diferentes tipos de embarques nas estações.

In [ ]:

# Classe para análise do dataset de Embarque
class EmbarkationAnalysis:
    def __init__(self, file_path):
        self.df_embarque = pd.read_csv(file_path, sep=',')
        self.df_embarque.columns = self.df_embarque.columns.str.replace('"', '')

    def preprocess_data(self):
        self.df_embarque['TX_MOVIMENTO'] = self.df_embarque['TX_MOVIMENTO'].astype(str)
        self.df_embarque['TX_LANCAMENTO'] = self.df_embarque['TX_LANCAMENTO'].astype(str)

        print("Dados de Embarque Após Preprocessamento:")
        print(self.df_embarque.head())

    def univariate_analysis(self):
        lancamento_labels = ['Gratuidade - Bloqueio', 'Outro', 'Outros', 'Pagante']

        plt.figure(figsize=(10,6))
        ax = sns.countplot(x='TX_LANCAMENTO', data=self.df_embarque, palette='Set1')

        ax.set_xticklabels(lancamento_labels)

        plt.title('Distribuição dos Tipos de Lançamento')
        plt.xlabel('Tipo de Lançamento')
        plt.ylabel('Frequência')
        plt.show()

    def encode_categorical(self):
        le = LabelEncoder()
        self.df_embarque['TX_MOVIMENTO'] = le.fit_transform(self.df_embarque['TX_MOVIMENTO'])
        self.df_embarque['TX_LANCAMENTO'] = le.fit_transform(self.df_embarque['TX_LANCAMENTO'])

        print("Dados após Codificação Categórica (Embarque):")
        print(self.df_embarque[['TX_MOVIMENTO', 'TX_LANCAMENTO']].head())

    def pca_analysis(self):
        scaler = StandardScaler()
        scaled_data_emb = scaler.fit_transform(self.df_embarque[['TX_MOVIMENTO', 'TX_LANCAMENTO']])
        pca_emb = PCA(n_components=2)
        pca_result_emb = pca_emb.fit_transform(scaled_data_emb)

        plt.figure(figsize=(8,6))
        plt.scatter(pca_result_emb[:,0], pca_result_emb[:,1], c=self.df_embarque['TX_MOVIMENTO'], cmap='viridis', alpha=0.5)
        plt.colorbar(label='Tipo de Movimento')
        plt.title('PCA - Tipos de Embarque')
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.show()
        
    def pareto_analisys(self):
        tipo_embarque_counts = self.df_embarque['TX_LANCAMENTO'].value_counts()
        tipo_embarque_percent = 100 * tipo_embarque_counts.cumsum() / tipo_embarque_counts.sum()

        # Gráfico de Pareto
        fig, ax = plt.subplots(figsize=(10,6))
        ax.bar(tipo_embarque_counts.index, tipo_embarque_counts.values, color='green')
        ax2 = ax.twinx()
        ax2.plot(tipo_embarque_percent.index, tipo_embarque_percent.values, color='red', marker='o', linestyle='dashed')

        ax.set_title('Gráfico de Pareto - Tipos de Bilhete')
        ax.set_xlabel('Tipo de Bilhete')
        ax.set_ylabel('Quantidade de Validações')
        ax2.set_ylabel('Percentual Acumulado (%)')

        plt.show()

        
    def return_df(self):
        return self.df_embarque

In [ ]:
embarkation_analysis = EmbarkationAnalysis('DMO_ANL_VW_TIPO_EMBARQUE.csv')

### Pré-processamento

Essas colunas contêm dados numéricos sobre o tipo de bilhete e o movimento associado a cada tipo de bilhete. A análise descritiva permite observar valores médios e extremos, facilitando a identificação de padrões de embarque por tipo de bilhete.

In [ ]:
embarkation_analysis.preprocess_data()

### Análise Univariada

A coluna TX_LANCAMENTO contém informações sobre o tipo de lançamento de bilhete (pagante, gratuidade, bloqueado, etc.). A visualização da distribuição de lançamentos permite analisar quais são os tipos de bilhete mais utilizados pelos passageiros, o que pode influenciar decisões tarifárias e de política pública.

In [ ]:
embarkation_analysis.univariate_analysis()

- Hipótese: A maioria dos embarques é feita por passageiros pagantes, mas uma parte significativa envolve gratuidades para grupos específicos.

- Análise: O gráfico de barras mostra que a maioria dos embarques é feita por pagantes, enquanto gratuidades e outros tipos de lançamentos têm uma presença menor, mas ainda relevante.

- Conclusão: O sistema de transporte atende majoritariamente pagantes, mas também oferece um suporte importante para gratuidades, especialmente para idosos, deficientes e outros grupos sociais, o que deve ser considerado no planejamento de recursos.

In [ ]:
embarkation_analysis.encode_categorical()

### Análise de Componentes Principais (PCA)

A PCA é usada aqui para identificar quais são as principais variáveis que explicam a maior parte da variabilidade entre os diferentes tipos de bilhetes. Isso permite agrupar tipos de bilhetes com comportamento similar e entender como os diferentes tipos de movimentos se relacionam com o tipo de bilhete usado.

In [ ]:
embarkation_analysis.pca_analysis()

- Hipótese: A análise de componentes principais irá mostrar que as categorias de embarque, como tipos de movimento, variam significativamente em relação ao tipo de lançamento.

- Análise: O gráfico de PCA mostra uma separação clara entre diferentes tipos de movimento, sugerindo que o tipo de lançamento (pagante, gratuidade, bloqueio) tem um impacto significativo na variação entre os dados.

- Conclusão: A variabilidade no dataset de embarque pode ser explicada em grande parte pelos tipos de movimento e lançamentos, indicando que diferentes perfis de passageiros utilizam o serviço de maneira diferenciada.

### Gráfico de Pareto

O gráfico de Pareto é utilizado para destacar quais tipos de bilhete representam a maior parte das validações. Isso é importante para entender quais categorias de passageiros (pagantes, gratuidades) têm mais impacto no volume total de embarques, ajudando a CPTM a tomar decisões baseadas em dados sobre como ajustar a oferta e política tarifária.

In [ ]:
embarkation_analysis.pareto_analisys()

- Hipótese: A maioria dos embarques é realizada por um número limitado de tipos de bilhetes, indicando que a maior parte da receita da CPTM provém de bilhetes pagantes, enquanto os demais representam uma pequena parcela das validações.

- Análise: O gráfico de Pareto mostra que cerca de 60% dos bilhetes validados pertencem a um único tipo de bilhete (possivelmente o tipo pagante). Os outros tipos de bilhetes têm menor representatividade, com cerca de 40% dos embarques sendo distribuídos entre os outros bilhetes. Isso sugere que uma grande parte do sistema de bilhetagem está concentrada em um ou dois tipos de bilhetes principais.

- Conclusão: A análise de Pareto revela que a maior parte dos embarques da CPTM provém de bilhetes pagantes, o que confirma a importância de uma política de tarifação bem estabelecida para a saúde financeira da empresa. A presença de outros tipos de bilhetes, como gratuidades e tarifas reduzidas, também deve ser monitorada para garantir que os subsídios ou incentivos governamentais sejam adequados para manter o equilíbrio das operações. Além disso, essa análise pode ajudar a CPTM a ajustar suas estratégias de marketing e otimização de serviços para atender diferentes perfis de usuários, promovendo o uso do transporte público por passageiros que ainda não representam a maioria dos embarques.

## Transformando em Parquet


In [ ]:
from services.parquet import Conversor

conversor = Conversor()

conversor.df_to_parquet(validation_analysis.return_df(), "./periodo.parquet")

conversor.df_to_parquet(embarkation_analysis.return_df(), "./embarque.parquet")

## Upload para o S3

In [ ]:
from services.aws_conn import AwsConn

aws_conn = AwsConn()

aws_conn.send_to_s3("embarque.parquet", "big-data-DMO-ANL", "./embarque.parquet")

aws_conn.send_to_s3("periodo.parquet", "big-data-DMO-ANL", "./periodo.parquet")